In [1]:
import os
import cv2
import shutil
import deeplabcut
import numpy as np
import os

2023-05-02 15:52:19.738297: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-02 15:52:19.842985: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-05-02 15:52:20.153860: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/yramakrishna/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:/usr/local/cuda/lib64:
2023-05-02 15:52:20.153909: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic l

Loading DLC 2.3.1...


In [2]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes'

In [3]:
start_file  = '/home/yramakrishna/DeepLabCut/conda-environments/Codes/Ken_Test_Long.mp4'

In [4]:
dsv = deeplabcut.DownSampleVideo(start_file, width=320, height=240)

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [5]:
os.mkdir("final_videor")

In [6]:
os.chdir('final_videor')

In [7]:
#cropping video and Gamma correction
for i in range(6):
    # Load the video file
    video = cv2.VideoCapture(dsv)

    # Get the frame rate and total number of frames
    fps = int(video.get(cv2.CAP_PROP_FPS))
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    #*********************************************

    # Set the start and end points to crop out
    start_sec = 0  # Crop out the first x seconds
    end_sec = 0 # Crop out the last y seconds

    #Set gamma correction value
    gamma = (i+1)/2
    print(gamma)

    #**********************************************

    start_frame = int(start_sec * fps)
    end_frame = int((total_frames / fps) - end_sec) * fps

    # Set the video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter('CropGr'+str(i)+'.mp4', fourcc, fps, (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))))

    # Set the current frame number to the start frame
    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    # Loop through the frames and write them to the output video
    for i in range(start_frame, end_frame):
        ret, frame = video.read()
        if not ret:
            break

        # Normalize the pixel values to the range [0, 1] 
        frame_normalized = frame.astype(np.float32) / 255.0 

        # Apply gamma correction 
        frame_corrected = np.power(frame_normalized, 1/gamma) 

        # Scale the pixel values back to the range [0, 255]
        frame_scaled = (frame_corrected * 255.0).astype(np.uint8)

        out.write(frame_scaled)

    # Release the video objects
    video.release()
    out.release()
    cv2.destroyAllWindows()

0.5
1.0
1.5
2.0
2.5
3.0


In [8]:
os.getcwd()

'/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor'

In [10]:

fold = os.listdir('/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor')
file = fold[0]
bodyparts = ['Eyes_MidPoint', 'RightEye_Outer', 'RightEye_Top', 'RightEye_Bottom', 'RightEye_Inner', 'RightEye_Pupil', 'LeftEye_Outer', 'LeftEye_Top', 'LeftEye_Bottom', 'LeftEye_Inner', 'LeftEye_Pupil',
             'NostrilsTop_Centre', 'OutlineTop_Mid', 'RightNostrils_Top', 'RightNostrils_Bottom','LeftNostrils_Top', 'LeftNostrils_Bottom']
videotype = os.path.splitext(file)[-1].lstrip('.')  # or MOV, or avi, whatever you uploaded!
video_down = file

model_options = deeplabcut.create_project.modelzoo.Modeloptions
model_selection = 'primate_face'

project_name = 'DLC'
your_name = 'VK'

config_path, train_config_path = deeplabcut.create_pretrained_project(
    project_name,
    your_name,
    video_down,
    videotype=videotype,
    model=model_selection,
    analyzevideo=True,
    createlabeledvideo=False,
    copy_videos=True,
)

edits = {
    'dotsize': 1.5,  # size of the dots!
    'pcutoff': 0.4,  # the higher, the more conservative the plotting!
}
deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

project_path = os.path.dirname(config_path)

full_video_path = []
for i in fold:
    full_video_path.append(os.path.join(project_path,'videos', os.path.basename(i)))
        
# filter predictions (should already be done above 
deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

# re-create the video with your edits!
deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts, filtered=True)

Created "/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos"
Created "/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/labeled-data"
Created "/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/training-datasets"
Created "/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/dlc-models"
6  videos from the directory . were added to the project.
Copying the videos
/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr2.mp4
/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr1.mp4
/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr4.mp4
/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr5.mp4
/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DL

/home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/dlc-models/iteration-0/DLCMay2-trainset95shuffle1/train/pose_cfg.yaml
Analyzing video...
Using snapshot-1030000 for model /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/dlc-models/iteration-0/DLCMay2-trainset95shuffle1


/home/yramakrishna/anaconda3/lib/python3.9/site-packages/tensorflow/python/keras/engine/base_layer_v1.py:1694: UserWarning: `layer.apply` is deprecated and will be removed in a future version. Please use `layer.__call__` method instead.
  warnings.warn('`layer.apply` is deprecated and '
2023-05-02 15:56:31.011107: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 15:56:31.024045: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 15:56:31.024211: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-05-02 15:56:31.024532: I tensorflow/core/platfo

Analyzing all the videos in the directory...
Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr5.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr5.mp4
Duration of video [s]:  457.0 , recorded with  29.0 fps!
Overall # of frames:  13253  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


  0%|                                                 | 0/13253 [00:00<?, ?it/s]2023-05-02 15:56:41.931432: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 37.50MiB (rounded to 39321600)requested by op resnet_v1_50/conv1/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-02 15:56:41.931503: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-02 15:56:41.931517: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 123, Chunks in use: 123. 30.8KiB allocated for chunks. 30.8KiB in use in bin. 14.7KiB client-requested in use in bin.
2023-05-02 15:56:41.931525: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 66, Chunks in use: 66. 33.0KiB allocated for c

...

Starting to analyze %  /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr1.mp4
Loading  /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr1.mp4
Duration of video [s]:  457.0 , recorded with  29.0 fps!
Overall # of frames:  13253  found with (before cropping) frame dimensions:  320 240
Starting to extract posture


  0%|                                                 | 0/13253 [00:00<?, ?it/s]2023-05-02 15:57:33.288539: W tensorflow/core/common_runtime/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 37.50MiB (rounded to 39321600)requested by op resnet_v1_50/conv1/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-05-02 15:57:33.288608: I tensorflow/core/common_runtime/bfc_allocator.cc:1033] BFCAllocator dump for GPU_0_bfc
2023-05-02 15:57:33.288623: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (256): 	Total Chunks: 153, Chunks in use: 153. 38.2KiB allocated for chunks. 38.2KiB in use in bin. 14.9KiB client-requested in use in bin.
2023-05-02 15:57:33.288631: I tensorflow/core/common_runtime/bfc_allocator.cc:1040] Bin (512): 	Total Chunks: 66, Chunks in use: 66. 33.0KiB allocated for c

The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr2.mp4
No unfiltered data file found in /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos for video CropGr2 and scorer DLC_resnet50_DLCMay2shuffle1_1030000.
Filtering with median model /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr1.mp4
No unfiltered data file found in /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos for video CropGr1 and scorer DLC_resnet50_DLCMay2shuffle1_1030000.
Filtering with median model /home/yramak

Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr2.mp4Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr4.mp4Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr5.mp4Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr1.mp4Starting to process video: /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr0.mp4



Loading /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr2.mp4 and data.Loading /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/videos/CropGr5.mp4 and data.Loading /home/yramakrishna/DeepLabCut/conda-environments/Codes/final_videor/DLC-VK-2023-05-02/vid

[False, False, False, False, False, False]

In [ ]:
# project_name = 'DLC_GazeXBI'
# your_name = 'anc'

# config_path, train_config_path = deeplabcut.create_pretrained_project(
#     project_name,
#     your_name,
#     video_down,
#     videotype=videotype,
#     model=model_selection,
#     analyzevideo=True,
#     createlabeledvideo=False,
#     copy_videos=True,
# )



In [ ]:
# edits = {
#     'dotsize': 3,  # size of the dots!
#     'pcutoff': 0.1,  # the higher, the more conservative the plotting!
# }
# deeplabcut.auxiliaryfunctions.edit_config(config_path, edits)

# project_path = os.path.dirname(config_path)
# full_video_path = os.path.join(
#     project_path,
#     'videos',
#     os.path.basename(video_down),
# )

# # filter predictions (should already be done above ;):
# deeplabcut.filterpredictions(config_path, full_video_path, videotype=videotype)

# # re-create the video with your edits!
# deeplabcut.create_labeled_video(config_path, full_video_path, videotype=videotype, displayedbodyparts=bodyparts,
#                                 filtered=True)


In [ ]:
# shutil.move('/home/yramakrishna/DeepLabCut/conda-environments/final_video/DLC_GazeXBI-anc-2023-03-21/videos/CropGrDLC_resnet50_DLC_GazeXBIMar21shuffle1_1030000_filtered_labeled.mp4', '/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar21shuffle1_1030000_filtered_labeled.mp4')



In [ ]:
# try:
#     shutil.rmtree('/home/yramakrishna/DeepLabCut/conda-environments/final_video')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# try:
#     os.remove('/home/yramakrishna/DeepLabCut/conda-environments/Ken_Test_Longdownsampled.mp4')
# except OSError as e:
#     # If it fails, inform the user.
#     print("Error: %s - %s." % (e.filename, e.strerror))

In [ ]:
# cap = cv2.VideoCapture("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# ret, frame = cap.read()
# while(1):
#    ret, frame = cap.read()
#    cv2.imshow('frame',frame)
#    if cv2.waitKey(1) & 0xFF == ord('q') or ret==False :
#        cap.release()
#        cv2.destroyAllWindows()
#        break
#    cv2.imshow('frame',frame)

In [ ]:
# from moviepy.editor import *
# import pygame
 
# clip = VideoFileClip('/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4')
# clip.preview()
 
# pygame.quit()

In [ ]:
# import tkinter as tk
# from tkVideoPlayer import TkinterVideo

# root = tk.Tk()

# videoplayer = TkinterVideo(master=root, scaled=True)
# videoplayer.load(r"/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4")
# videoplayer.pack(expand=True, fill="both")

# videoplayer.play() # play the video

# root.mainloop()

In [ ]:
# from tkinter import *
# from tkvideo import tkvideo

# root = Tk()
# my_label = Label(root)
# my_label.pack()
# player = tkvideo("/home/yramakrishna/DeepLabCut/conda-environments/CropGrDLC_resnet50_DLC_GazeXBIMar20shuffle1_1030000_filtered_labeled.mp4", my_label, loop = 1, size = (1280,720))
# player.play()

# root.mainloop()